In [ ]:
!pip install datasets evaluate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from typing import Optional, Union
from datasets import load_dataset
from dataclasses import dataclass
import evaluate
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForMultipleChoice, get_scheduler, AutoConfig, AutoModel
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from tqdm import tqdm
import argparse

import numpy as np
import scipy as sp

import torch.nn as nn
import torch.nn.functional as F
import argparse
import json
import os
import sys
import random
import wandb


import random

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

SEED = 595
set_seed(595)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")



In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/EECS595/Project/Verifiable-Coherent-NLU-main'

In [ ]:
DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(DRIVE_PATH)
print(os.listdir(DRIVE_PATH))

['README.md', 'requirements.txt', 'www', 'all_data', 'cache', 'saved_models', 'Verifiable-Coherent-NLU.ipynb', 'BERT_CE.ipynb', 'withBRET_CE.ipynb', 'project_fineTunePIQA.ipynb']


In [ ]:
import os
import huggingface_hub
import wandb
#from dotenv import load_dotenv



In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator to dynamically pad inputs for multiple choice tasks.

    This class helps to prepare batches of features for models that handle multiple choice tasks,
    ensuring that inputs are properly padded and reshaped before being passed to the model. The
    padding is applied dynamically based on the longest sequence in the batch, or according to
    the specified `max_length` and `pad_to_multiple_of` parameters.

    Attributes:
        tokenizer (AutoTokenizer): The tokenizer used to pad and process the input features.
        device (torch.device): The device (CPU or GPU) where the padded tensors should be sent.
        padding (Union[bool, str, PaddingStrategy]): The padding strategy. If `True`, pads all sequences to
                                                     the length of the longest one. If a string, defines a
                                                     specific padding type ('longest' or 'max_length').
        max_length (Optional[int]): The maximum length to pad sequences to, if specified.
        pad_to_multiple_of (Optional[int]): If provided, pads sequences to a multiple of this value.

    Methods:
        __call__(features, label_name="labels"):
            Pads and collates a batch of features for multiple choice tasks.
            Extracts the labels from the feature dictionary, flattens the features, applies dynamic padding,
            and reshapes the padded features back into the multiple choice format.
    """

    tokenizer: AutoTokenizer
    device: torch.device
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features, label_name="labels"):
        """
        Pads and collates a batch of features for the model, maintaining structure for multiple choice tasks.

        This method first extracts the labels from the input features and then flattens the feature structure
        for each option in the multiple choice task. After flattening, it applies padding to ensure all inputs
        in the batch have the same length. The padded batch is then reshaped back into the original multiple
        choice format and returned as a dictionary of tensors.

        Params:
            features (list of dict): A list of dictionaries, where each dictionary represents the input features
                                     (such as input_ids, attention_mask, etc.) for one example.
            label_name (str, optional): The key used to extract labels from the feature dictionaries. Default is "labels".

        Returns:
            batch (dict of torch.Tensor): A dictionary containing the padded and collated tensors for the batch,
                                          including the input_ids, attention_masks, etc., as well as the labels.
        """
        batch = {}

        ###############################################################
        #########         TODO: Your code starts here        ##########
        ###############################################################

        # Extract labels from the list of feature dictionaries and remove them from the feature dicts
        labels = [feature.pop(label_name) for feature in features]

        # Get the batch size (number of examples in the batch) and the number of options (solutions)
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])

        # Flatten the structure of features for padding.
        # Each feature dictionary contains multiple options for the task (e.g., multiple choices),
        # and we want to treat each option as a separate instance for padding purposes.
        # `flattened_features` is now a list of lists, so we flatten it to make it a single list of feature dicts.
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        # Apply padding to the flattened features.
        # This ensures that all the sequences (e.g., input_ids, attention_masks) in the batch
        # are padded to the same length. Padding can be customized by the parameters `padding`,
        # `max_length`, and `pad_to_multiple_of`.
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )


        # Reshape the padded batch back to its original structure.
        # After padding, we reshape the tensors to reflect the original batch structure (batch_size, num_solutions, seq_len).
        # For each tensor (input_ids, attention_mask, etc.), we view it as a 3D tensor where the first dimension
        # corresponds to the batch, the second dimension corresponds to the different options (solutions),
        # and the third dimension is the sequence length (after padding).
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Convert the labels into a tensor and move them to the specified device (e.g., GPU or CPU).
        labels = torch.tensor(labels, device=self.device,dtype=torch.long)
        batch["labels"] = labels
        batch = {k: v.to(self.device) for k, v in batch.items()}

        ###############################################################
        #########          TODO: Your code ends here         ##########
        ###############################################################

        return batch


In [ ]:
def load_piqa_data(tokenizer, device, params):
    """
    Loads and tokenizes the PIQA dataset, returning DataLoader objects for the train, validation, and test sets.
    The PIQA dataset contains physical commonsense reasoning questions with two possible solutions, and this function
    tokenizes the goal and solutions for multiple-choice classification tasks.

    Params:
        tokenizer (AutoTokenizer): The tokenizer to be used for tokenizing the dataset examples.
        device (torch.device): The device (CPU or GPU) to which the processed batches will be moved.
        params (Namespace or dict): A collection of parameters, including the dataset name and batch size.

    Returns:
        tuple: A tuple of DataLoader objects for the train, validation, and test sets.
    """

    num_solutions = 2  # Each PIQA example has 2 possible solutions (sol1 and sol2).

    def tokenize_function(examples):
        """
        Tokenizes the inputs (goal + solutions) for multiple choice classification tasks.

        Each goal in the dataset is paired with two potential solutions (sol1 and sol2).
        The function prepares input sequences by concatenating the goal with each solution, tokenizes them,
        and returns the tokenized results for both solutions.

        Params:
            examples (dict): A batch of dataset examples containing 'goal', 'sol1', 'sol2', and 'label' fields.

        Returns:
            tokenized_dict (dict): A dictionary of tokenized inputs, with solutions grouped into pairs.
        """

        tokenized_dict = {}
        ###############################################################
        #########         TODO: Your code starts here        ##########
        ###############################################################

        # Create a list where each goal is repeated for both solutions.
        ######goals = [goal for goal in examples["goal"] for _ in range(num_solutions)]
        goals = [goal for goal in examples["goal"] for _ in range(num_solutions)]


        # Create a list of solution pairs (sol1 and sol2).
        #####solutions = [f"{sol1} {sol2}" for sol1, sol2 in zip(examples["sol1"], examples["sol2"])]
        #solutions = examples["sol1"] + examples["sol2"]
        solutions = []
        for sol1, sol2 in zip(examples["sol1"], examples["sol2"]):
          solutions.extend([sol1, sol2])


        # Flatten the goals and solutions into single lists for tokenization.
        #flattened_goals_and_solutions = [goal + " " + solution for goal, solution in zip(goals, solutions)]

        # Tokenize the pairs of goal and solution sentences.

        tokenized_dict = tokenizer(
            goals,
            solutions,
            truncation=True,
            padding="max_length",
            max_length=256,  # Adjust max_length as appropriate
            return_tensors="pt"
            )

        # Group the tokenized results back into pairs, corresponding to the number of solutions.
        for k, v in tokenized_dict.items():
          tokenized_dict[k] = v.view(len(examples['goal']), num_solutions, -1)
          #tokenized_dict[k] = v.view(-1, num_solutions, v.shape[-1])

        tokenized_dict["label"] = torch.tensor(examples["label"], dtype=torch.long)

        ###############################################################
        #########          TODO: Your code ends here         ##########
        ###############################################################
        return tokenized_dict

    # Load the dataset from the provided parameters.
    dataset = load_dataset(params.dataset)

    # Tokenize and process the dataset using the `tokenize_function`.
    tokenized_datasets = dataset.map(tokenize_function, batched=True)  # Apply tokenization to the entire dataset.
    tokenized_datasets = tokenized_datasets.remove_columns(["goal", "sol1", "sol2"])  # Remove unnecessary columns.
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")  # Rename 'label' to 'labels' for consistency.
    tokenized_datasets.set_format("torch")  # Set the dataset format to PyTorch tensors.

    # Initialize a data collator to dynamically pad the inputs during batch creation.
    data_collator = DataCollatorForMultipleChoice(tokenizer, device)

    # Create DataLoader objects for the train, validation, and test sets.
    train_dataloader = DataLoader(
        tokenized_datasets["train"],  # Training dataset.
        shuffle=True,  # Shuffle the training data.
        batch_size=params.batch_size,  # Use the batch size from the params.
        collate_fn=data_collator  # Use the custom data collator to pad and batch the data.
    )

    eval_dataloader = DataLoader(
        tokenized_datasets["validation"],  # Validation dataset.
        batch_size=params.batch_size,  # Batch size for validation.
        collate_fn=data_collator  # Use the custom data collator.
    )

    test_dataloader = DataLoader(
        tokenized_datasets["test"],  # Test dataset.
        batch_size=params.batch_size,  # Batch size for testing.
        collate_fn=data_collator  # Use the custom data collator.
    )

    # Return the DataLoader objects for train, validation, and test sets.
    return train_dataloader, eval_dataloader, test_dataloader


In [ ]:
def finetune(model, train_dataloader, eval_dataloader, params):
    """
    Fine-tunes the BERT model for the PIQA task.

    This function trains a BERT-based model on a multiple-choice classification task (e.g., PIQA) using
    the provided training and evaluation data. It utilizes an optimizer, learning rate scheduler,
    and tracks the validation accuracy after each epoch.

    Params:
        model (PreTrainedModel): The pre-trained BERT model to be fine-tuned.
        train_dataloader (DataLoader): A DataLoader providing batches of training data.
        eval_dataloader (DataLoader): A DataLoader providing batches of validation data.
        params (Namespace or dict): A collection of parameters, including the learning rate, number of epochs,
                                    and other training configurations (e.g., num_warmup_steps).

    Returns:
        model (PreTrainedModel): The fine-tuned model.
    """

    # Calculate the total number of training steps (epochs * batches per epoch).
    num_training_steps = params.num_epochs * len(train_dataloader)

    # Initialize the optimizer (AdamW) with the model's parameters and learning rate.
    optimizer = AdamW(model.parameters(), lr=params.lr)

    # Set up the learning rate scheduler for controlling the learning rate dynamically during training.
    # We use a linear schedule with warmup steps.
    lr_scheduler = get_scheduler(
        name="linear",  # Linear learning rate decay.
        optimizer=optimizer,  # Optimizer whose learning rate will be scheduled.
        num_warmup_steps=params.num_warmup_steps,  # Number of warmup steps before decay begins.
        num_training_steps=num_training_steps  # Total number of training steps.
    )

    # Load the evaluation metric to be used during validation (accuracy in this case).
    metric = evaluate.load("accuracy")

    # Initialize a progress bar to visually track training progress over the total number of steps.
    progress_bar = tqdm(range(num_training_steps))

    # Training loop over the specified number of epochs.
    for epoch in range(params.num_epochs):

        # Set the model to training mode (enables dropout and gradient computation).
        model.train()
        for batch in train_dataloader:

            ###############################################################
            #########         TODO: Your code starts here        ##########
            ###############################################################
            # Forward pass: run the model on the batch and compute the loss.
            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            # Backpropagate the loss to compute gradients.
            loss.backward()

            # Update model parameters using the optimizer.
            optimizer.step()

            # Update the learning rate according to the schedule.
            lr_scheduler.step()

            # Reset the gradients for the next step.
            optimizer.zero_grad()

            ###############################################################
            #########          TODO: Your code ends here         ##########
            ###############################################################

            # Update the progress bar for each step.
            progress_bar.update(1)


        avg_loss = loss.item() / len(train_dataloader)
        print(f"Average Training Loss: {avg_loss:.4f}")

        # Validation loop: evaluate the model on the validation data after each epoch.
        model.eval()  # Set the model to evaluation mode (disables dropout and gradient computation).
        for batch in eval_dataloader:
            with torch.no_grad():  # Disable gradient calculation for efficiency.
                #batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**batch)  # Forward pass on the validation batch.

            logits = outputs.logits  # Extract logits (predicted scores for each class).
            predictions = torch.argmax(logits, dim=-1)  # Take the argmax to get predicted classes.
            # Add the predictions and true labels to the accuracy metric for evaluation.
            metric.add_batch(predictions=predictions, references=batch["labels"])

        # Compute the validation accuracy after the epoch.
        score = metric.compute()
        print(f'Validation Accuracy: {score["accuracy"]}')  # Print the validation accuracy.

    # Return the fine-tuned model.
    return model

In [ ]:
def test(model, test_dataloader, prediction_save='bert_predictions.torch'):
    """
    Evaluates the fine-tuned model on the test dataset and saves the predictions.

    This function runs the model on the test dataset, computes accuracy, and saves the model's predictions
    to a file. It uses a DataLoader to load the test data in batches, computes predictions without
    gradient calculations, and stores them in a list. The final accuracy is printed and the predictions
    are saved to disk.

    Params:
        model (PreTrainedModel): The fine-tuned model to be evaluated.
        test_dataloader (DataLoader): A DataLoader providing batches of test data.
        prediction_save (str, optional): The file path where predictions will be saved.
                                         Default is 'bert_predictions.torch'.

    Returns:
        None
    """

    # Load the accuracy metric to evaluate the test predictions.
    metric = evaluate.load("accuracy")

    # Set the model to evaluation mode (disables dropout and gradient computation).
    model.eval()

    # Initialize a list to store predictions.
    predictions = []


    # Loop over batches of test data from the test dataloader.
    for batch in test_dataloader:

        with torch.no_grad():  # Disable gradient computation for efficiency during testing.
            # Forward pass: get the model's outputs for the test batch.

            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

        # Extract logits (model's predicted scores for each class).
        logits = outputs.logits

        # Get the predicted class by taking the argmax of the logits (the class with the highest score).
        preds = torch.argmax(logits, dim=-1)

        # Add predictions and labels to the accuracy metric for evaluation.
        metric.add_batch(predictions=preds, references=batch["labels"])

        # Convert the predictions to CPU and extend the list with the new batch predictions.
        predictions.extend(preds.cpu().numpy())

    # Compute the final accuracy on the test set.
    score = metric.compute()

    # Print the test accuracy.
    print(f'Test Accuracy: {score["accuracy"]}')

    # Save the predictions to a file in the specified format.
    torch.save(predictions, prediction_save)


In [ ]:
def main(params):
    """
    Main function that handles tokenization, fine-tuning, and testing of the model on the PIQA dataset.

    This function:
    1. Sets up the device for computation (CPU or GPU).
    2. Loads the tokenizer and data using the provided parameters.
    3. Loads a pre-trained model for multiple choice tasks.
    4. Fine-tunes the model using the training and validation datasets.
    5. Evaluates the fine-tuned model on the test dataset and prints the results.

    Params:
        params (Namespace or dict): A collection of parameters, including the model name, number of epochs,
                                    batch size, learning rate, and dataset configuration.

    Returns:
        None
    """

    # Select the device (either CPU or GPU if available).
    device = torch.device("cpu")  # Default to CPU.
    if torch.cuda.is_available():  # Check if CUDA-enabled GPU is available.
        device = torch.device("cuda")
    print('Using device:', device)

    # Load the tokenizer for the model specified in the params (e.g., BERT tokenizer).
    tokenizer = AutoTokenizer.from_pretrained(params.model)

    # Load and tokenize the dataset (PIQA) and return DataLoaders for training, evaluation, and testing.
    train_dataloader, eval_dataloader, test_dataloader = load_piqa_data(tokenizer, device, params)

    # Load the pre-trained multiple choice model from Hugging Face's model hub.
    model = AutoModelForMultipleChoice.from_pretrained(params.model)

    # Move the model to the selected device (GPU or CPU).
    model.to(device)

    # Fine-tune the pre-trained model on the training dataset and validate it using the evaluation dataset.
    model = finetune(model, train_dataloader, eval_dataloader, params)

    model.save_pretrained(os.path.join(DRIVE_PATH, 'saved_models/', params.output_dir))
    tokenizer.save_pretrained(os.path.join(DRIVE_PATH, 'saved_models/', params.output_dir))

    # Test the fine-tuned model on the test dataset and print accuracy results.
    test(model, eval_dataloader)




In [ ]:
if __name__ == "__main__":
    batch_size = 8
    num_epochs = 10
    lr = 1e-5
    num_warmup_steps = 2016

    ###############################################################
    #########          TODO: Your code ends here         ##########
    ###############################################################

    # Argument parser for command line execution
    parser = argparse.ArgumentParser(description="Finetune Large Bert for PIQA Task")
    parser.add_argument("--output_dir", type=str, default="finetunedBert_large_PIQA/", help="Output directory for model")
    parser.add_argument("--dataset", type=str, default="piqa", help="Dataset name")
    parser.add_argument("--model", type=str, default="bert-large-uncased", help="Pretrained model name")
    parser.add_argument("--batch_size", type=int, default=batch_size, help="Batch size for training")
    parser.add_argument("--num_epochs", type=int, default=num_epochs, help="Number of training epochs")
    parser.add_argument("--lr", type=float, default=lr, help="Learning rate for AdamW optimizer")
    parser.add_argument("--num_warmup_steps", type=int, default=num_warmup_steps, help="Number of warmup steps for learning rate scheduler")

    # Parse arguments and run the main function
    params, unknown = parser.parse_known_args()
    main(params)


Using device: cuda


Map:   0%|          | 0/3084 [00:00<?, ? examples/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

  0%|          | 0/20150 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.

 10%|█         | 2015/20150 [16:29<2:14:50,  2.24it/s]

Average Training Loss: 0.0005
Validation Accuracy: 0.5076169749727966



 20%|██        | 4030/20150 [33:37<1:57:32,  2.29it/s]

Average Training Loss: 0.0003
Validation Accuracy: 0.5038084874863983



 24%|██▍       | 4901/20150 [41:23<2:04:57,  2.03it/s]